## 🔧 1. Setup & Check GPU

In [ ]:
# Install packages jika belum ada
# !pip install torch transformers scikit-learn pandas numpy matplotlib seaborn tqdm pynvml

import torch
import sys
import platform
import subprocess

print('=' * 60)
print('🖥️  SYSTEM INFO')
print('=' * 60)
print(f'Python: {sys.version}')
print(f'PyTorch: {torch.__version__}')
print(f'Platform: {platform.system()} {platform.release()}')
print()

# Check GPU
if torch.cuda.is_available():
    print('✅ CUDA is available!')
    print(f'   GPU: {torch.cuda.get_device_name(0)}')
    print(f'   CUDA Version: {torch.version.cuda}')
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f'   Total Memory: {gpu_memory:.1f} GB')
    device = torch.device('cuda')
    
    # Check nvidia-smi untuk power info
    try:
        result = subprocess.run(['nvidia-smi', '--query-gpu=power.limit,power.draw', '--format=csv,noheader,nounits'], 
                               capture_output=True, text=True)
        if result.returncode == 0:
            power_info = result.stdout.strip().split(',')
            print(f'   Power Limit: {float(power_info[0].strip()):.0f} W')
            print(f'   Current Power: {float(power_info[1].strip()):.1f} W')
    except:
        print('   Power info: nvidia-smi not available')
else:
    print('⚠️ CUDA not available, using CPU')
    print('   Training will be SLOW!')
    device = torch.device('cpu')

print(f'\n🎮 Using device: {device}')

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from tqdm.auto import tqdm
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import BertTokenizer, BertModel, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_recall_fscore_support, 
    classification_report, confusion_matrix, f1_score
)
from sklearn.utils import resample
import random
import os
import copy
import json
from datetime import datetime
import time
import subprocess

warnings.filterwarnings('ignore')

# =====================================================
# GPU MONITORING FUNCTIONS
# =====================================================
def get_gpu_memory_usage():
    """Get current GPU memory usage in MB"""
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated(0) / 1024**2
        reserved = torch.cuda.memory_reserved(0) / 1024**2
        return allocated, reserved
    return 0, 0

def get_gpu_power():
    """Get current GPU power draw in Watts"""
    try:
        result = subprocess.run(
            ['nvidia-smi', '--query-gpu=power.draw', '--format=csv,noheader,nounits'],
            capture_output=True, text=True
        )
        if result.returncode == 0:
            return float(result.stdout.strip())
    except:
        pass
    return 0.0

def get_gpu_stats():
    """Get comprehensive GPU stats"""
    if not torch.cuda.is_available():
        return None
    
    try:
        result = subprocess.run(
            ['nvidia-smi', '--query-gpu=memory.used,memory.total,power.draw,temperature.gpu,utilization.gpu', 
             '--format=csv,noheader,nounits'],
            capture_output=True, text=True
        )
        if result.returncode == 0:
            values = [v.strip() for v in result.stdout.strip().split(',')]
            return {
                'memory_used_mb': float(values[0]),
                'memory_total_mb': float(values[1]),
                'power_draw_w': float(values[2]),
                'temperature_c': float(values[3]),
                'gpu_util_percent': float(values[4])
            }
    except:
        pass
    return None

class GPUMonitor:
    """Monitor GPU usage during training"""
    def __init__(self):
        self.reset()
    
    def reset(self):
        self.memory_samples = []
        self.power_samples = []
        self.start_time = None
        self.end_time = None
        self.total_energy_wh = 0
    
    def start(self):
        self.reset()
        self.start_time = time.time()
        if torch.cuda.is_available():
            torch.cuda.reset_peak_memory_stats()
    
    def sample(self):
        """Take a sample of current GPU stats"""
        if torch.cuda.is_available():
            mem_allocated = torch.cuda.memory_allocated(0) / 1024**2
            self.memory_samples.append(mem_allocated)
            
            power = get_gpu_power()
            if power > 0:
                self.power_samples.append(power)
    
    def stop(self):
        self.end_time = time.time()
        
        # Calculate total energy consumption
        if self.power_samples and self.start_time:
            duration_hours = (self.end_time - self.start_time) / 3600
            avg_power = np.mean(self.power_samples)
            self.total_energy_wh = avg_power * duration_hours
    
    def get_summary(self):
        """Get summary of GPU usage"""
        summary = {
            'duration_seconds': 0,
            'duration_formatted': '0:00:00',
            'peak_memory_mb': 0,
            'avg_memory_mb': 0,
            'avg_power_w': 0,
            'max_power_w': 0,
            'total_energy_wh': 0,
            'total_energy_kwh': 0
        }
        
        if self.start_time and self.end_time:
            duration = self.end_time - self.start_time
            summary['duration_seconds'] = duration
            hours, remainder = divmod(int(duration), 3600)
            minutes, seconds = divmod(remainder, 60)
            summary['duration_formatted'] = f'{hours}:{minutes:02d}:{seconds:02d}'
        
        if torch.cuda.is_available():
            summary['peak_memory_mb'] = torch.cuda.max_memory_allocated(0) / 1024**2
        
        if self.memory_samples:
            summary['avg_memory_mb'] = np.mean(self.memory_samples)
        
        if self.power_samples:
            summary['avg_power_w'] = np.mean(self.power_samples)
            summary['max_power_w'] = np.max(self.power_samples)
            summary['total_energy_wh'] = self.total_energy_wh
            summary['total_energy_kwh'] = self.total_energy_wh / 1000
        
        return summary

# Initialize GPU monitor
gpu_monitor = GPUMonitor()

# Reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)
print('✓ Libraries imported')
print('✓ GPU monitoring functions defined')
print(f'✓ Device: {device}')

## 📊 2. Load Data

In [ ]:
# Load data dari folder data/
DATA_PATH = 'data/gojek_reviews_final_augmented.csv'

if not os.path.exists(DATA_PATH):
    print(f'❌ File tidak ditemukan: {DATA_PATH}')
    print('\n📁 Files yang ada:')
    if os.path.exists('data'):
        for f in os.listdir('data'):
            print(f'   - data/{f}')
else:
    df = pd.read_csv(DATA_PATH)
    
    print('=' * 60)
    print('📊 DATA OVERVIEW')
    print('=' * 60)
    print(f'Total samples: {len(df):,}')
    print(f'Columns: {df.columns.tolist()}')
    print(f'\n📈 Sentiment Distribution:')
    print(df['sentiment'].value_counts())
    
    # Visualize
    fig, ax = plt.subplots(1, 1, figsize=(8, 4))
    colors = {'negative': '#e74c3c', 'neutral': '#95a5a6', 'positive': '#2ecc71'}
    counts = df['sentiment'].value_counts()
    bars = ax.bar(counts.index, counts.values, color=[colors[s] for s in counts.index])
    ax.set_title('Sentiment Distribution')
    ax.set_ylabel('Count')
    for bar, count in zip(bars, counts.values):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 50, 
                str(count), ha='center', fontweight='bold')
    plt.tight_layout()
    plt.show()

## ⚖️ 3. Prepare Data

In [ ]:
# Check if data is already balanced
counts = df['sentiment'].value_counts()
min_count = counts.min()
max_count = counts.max()

if (max_count - min_count) / max_count < 0.1:
    print('✓ Data sudah balanced!')
    df_balanced = df.copy()
else:
    print('⚠️ Melakukan undersampling...')
    df_balanced = pd.DataFrame()
    for sentiment in ['negative', 'neutral', 'positive']:
        df_class = df[df['sentiment'] == sentiment]
        df_sampled = resample(df_class, replace=False, n_samples=min_count, random_state=42)
        df_balanced = pd.concat([df_balanced, df_sampled])
    df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

print(f'\n📊 Data untuk training: {len(df_balanced):,} samples')
print(df_balanced['sentiment'].value_counts())

In [ ]:
# Label mapping
LABEL_MAP = {'negative': 0, 'neutral': 1, 'positive': 2}
LABEL_NAMES = ['negative', 'neutral', 'positive']
NUM_CLASSES = 3

df_balanced['label'] = df_balanced['sentiment'].map(LABEL_MAP)

# Split: 70% train, 15% val, 15% test
train_df, temp_df = train_test_split(
    df_balanced, test_size=0.3, random_state=42, stratify=df_balanced['label']
)
val_df, test_df = train_test_split(
    temp_df, test_size=0.5, random_state=42, stratify=temp_df['label']
)

print('=' * 60)
print('📂 DATA SPLITS')
print('=' * 60)
print(f'Train: {len(train_df):,} samples ({len(train_df)/len(df_balanced)*100:.1f}%)')
print(f'Val:   {len(val_df):,} samples ({len(val_df)/len(df_balanced)*100:.1f}%)')
print(f'Test:  {len(test_df):,} samples ({len(test_df)/len(df_balanced)*100:.1f}%)')

## ⚙️ 4. Configuration

In [ ]:
# === HYPERPARAMETERS ===
# Optimized untuk menghindari overfitting

CONFIG = {
    # Model
    'model_name': 'indobenchmark/indobert-base-p1',
    'max_length': 128,
    'num_classes': NUM_CLASSES,
    
    # Training
    'batch_size': 32,
    'epochs': 30,
    'learning_rate': 5e-6,  # Sangat kecil untuk hindari overfitting
    
    # Anti-Overfitting
    'dropout_rate': 0.6,
    'attention_dropout': 0.3,
    'weight_decay': 0.05,
    'label_smoothing': 0.2,
    'warmup_ratio': 0.15,
    'max_grad_norm': 0.5,
    'early_stopping_patience': 7,
    
    # Data Augmentation
    'word_dropout_prob': 0.2,
    
    # Layer Freezing
    'freeze_layers': 9,  # Freeze 9 dari 12 layer
    
    # R-Drop
    'rdrop_alpha': 0.5,
}

print('=' * 60)
print('⚙️  CONFIGURATION')
print('=' * 60)
for key, value in CONFIG.items():
    print(f'{key}: {value}')

## 📦 5. Dataset & DataLoader

In [ ]:
# Load tokenizer
print('Loading tokenizer...')
tokenizer = BertTokenizer.from_pretrained(CONFIG['model_name'])
print(f'✓ Tokenizer loaded: {CONFIG["model_name"]}')

class SentimentDataset(Dataset):
    """Dataset dengan augmentation"""
    
    def __init__(self, texts, labels, tokenizer, max_length=128, 
                 augment=False, word_dropout_prob=0.2):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.augment = augment
        self.word_dropout_prob = word_dropout_prob
    
    def __len__(self):
        return len(self.texts)
    
    def _augment_text(self, text):
        if not self.augment:
            return text
        
        text = str(text)
        words = text.split()
        
        if len(words) <= 3:
            return text
        
        aug_type = random.random()
        
        if aug_type < 0.3:
            # Word dropout
            words = [w for w in words if random.random() > self.word_dropout_prob]
        elif aug_type < 0.5:
            # Word swap
            if len(words) > 2:
                idx = random.randint(0, len(words) - 2)
                words[idx], words[idx + 1] = words[idx + 1], words[idx]
        elif aug_type < 0.7:
            # Random deletion
            if len(words) > 4:
                del_idx = random.randint(0, len(words) - 1)
                words.pop(del_idx)
        
        return ' '.join(words) if words else text
    
    def __getitem__(self, idx):
        text = self._augment_text(self.texts[idx])
        
        encoding = self.tokenizer.encode_plus(
            str(text),
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(self.labels[idx], dtype=torch.long)
        }

In [ ]:
# Create datasets
train_dataset = SentimentDataset(
    train_df['content_clean'].values,
    train_df['label'].values,
    tokenizer,
    max_length=CONFIG['max_length'],
    augment=True,
    word_dropout_prob=CONFIG['word_dropout_prob']
)

val_dataset = SentimentDataset(
    val_df['content_clean'].values,
    val_df['label'].values,
    tokenizer,
    max_length=CONFIG['max_length'],
    augment=False
)

test_dataset = SentimentDataset(
    test_df['content_clean'].values,
    test_df['label'].values,
    tokenizer,
    max_length=CONFIG['max_length'],
    augment=False
)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=CONFIG['batch_size'], shuffle=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=CONFIG['batch_size'], shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=CONFIG['batch_size'], shuffle=False)

print(f'\n✓ Datasets created:')
print(f'  Train: {len(train_dataset)} samples, {len(train_loader)} batches')
print(f'  Val:   {len(val_dataset)} samples, {len(val_loader)} batches')
print(f'  Test:  {len(test_dataset)} samples, {len(test_loader)} batches')

## 🧠 6. Model

In [ ]:
class IndoBERTSentimentClassifier(nn.Module):
    """
    IndoBERT dengan regularisasi maksimal:
    - Freeze 9/12 layer BERT
    - Multiple dropout
    - Simple classifier
    """
    
    def __init__(self, model_name, num_classes, dropout_rate=0.6, 
                 attention_dropout=0.3, freeze_layers=9):
        super(IndoBERTSentimentClassifier, self).__init__()
        
        # Load pretrained BERT
        self.bert = BertModel.from_pretrained(model_name)
        self.hidden_size = self.bert.config.hidden_size
        
        # Freeze embeddings
        for param in self.bert.embeddings.parameters():
            param.requires_grad = False
        
        # Freeze first N encoder layers
        for i in range(freeze_layers):
            for param in self.bert.encoder.layer[i].parameters():
                param.requires_grad = False
        
        # Add dropout to attention in unfrozen layers
        for i in range(freeze_layers, 12):
            self.bert.encoder.layer[i].attention.self.dropout = nn.Dropout(attention_dropout)
            self.bert.encoder.layer[i].attention.output.dropout = nn.Dropout(attention_dropout)
        
        print(f'✓ Froze embeddings and first {freeze_layers} encoder layers')
        print(f'  Only layers {freeze_layers}-11 are trainable (3 layers)')
        
        # Regularization
        self.dropout1 = nn.Dropout(dropout_rate)
        self.dropout2 = nn.Dropout(dropout_rate)
        self.layer_norm = nn.LayerNorm(self.hidden_size)
        
        # Simple classifier
        self.fc = nn.Linear(self.hidden_size, num_classes)
        
        # Initialize
        nn.init.xavier_uniform_(self.fc.weight)
        nn.init.zeros_(self.fc.bias)
    
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        
        pooled_output = outputs.pooler_output
        
        x = self.layer_norm(pooled_output)
        x = self.dropout1(x)
        
        if self.training:
            x = self.dropout2(x)
        
        logits = self.fc(x)
        
        return logits

In [ ]:
# Initialize model
print('Loading IndoBERT model...')
model = IndoBERTSentimentClassifier(
    model_name=CONFIG['model_name'],
    num_classes=CONFIG['num_classes'],
    dropout_rate=CONFIG['dropout_rate'],
    attention_dropout=CONFIG['attention_dropout'],
    freeze_layers=CONFIG['freeze_layers']
).to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
frozen_params = total_params - trainable_params

print(f'\n✓ Model loaded to {device}')
print(f'  Total parameters: {total_params:,}')
print(f'  Trainable: {trainable_params:,} ({trainable_params/total_params*100:.1f}%)')
print(f'  Frozen: {frozen_params:,} ({frozen_params/total_params*100:.1f}%)')

## 📉 7. Loss, Optimizer & Scheduler

In [ ]:
# Loss function dengan label smoothing
criterion = nn.CrossEntropyLoss(label_smoothing=CONFIG['label_smoothing'])

# Optimizer - hanya untuk trainable parameters
no_decay = ['bias', 'LayerNorm.weight', 'layer_norm.weight']
trainable_params_list = [(n, p) for n, p in model.named_parameters() if p.requires_grad]

optimizer_grouped_parameters = [
    {
        'params': [p for n, p in trainable_params_list if not any(nd in n for nd in no_decay)],
        'weight_decay': CONFIG['weight_decay']
    },
    {
        'params': [p for n, p in trainable_params_list if any(nd in n for nd in no_decay)],
        'weight_decay': 0.0
    }
]

optimizer = AdamW(optimizer_grouped_parameters, lr=CONFIG['learning_rate'])

# Scheduler
total_steps = len(train_loader) * CONFIG['epochs']
warmup_steps = int(total_steps * CONFIG['warmup_ratio'])

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps
)

print(f'✓ Optimizer: AdamW (lr={CONFIG["learning_rate"]}, wd={CONFIG["weight_decay"]})')
print(f'✓ Scheduler: Linear warmup ({warmup_steps} warmup, {total_steps} total)')
print(f'✓ Loss: CrossEntropy with label_smoothing={CONFIG["label_smoothing"]}')

## 🏋️ 8. Training Functions

In [ ]:
def compute_kl_loss(p, q):
    """KL divergence for R-Drop"""
    p_loss = F.kl_div(F.log_softmax(p, dim=-1), F.softmax(q, dim=-1), reduction='batchmean')
    q_loss = F.kl_div(F.log_softmax(q, dim=-1), F.softmax(p, dim=-1), reduction='batchmean')
    return (p_loss + q_loss) / 2

def train_epoch(model, dataloader, criterion, optimizer, scheduler, device, 
                max_grad_norm, rdrop_alpha=0.5):
    """Train dengan R-Drop regularization"""
    model.train()
    total_loss = 0
    all_preds = []
    all_labels = []
    
    progress_bar = tqdm(dataloader, desc='Training', leave=False)
    
    for batch in progress_bar:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        
        optimizer.zero_grad()
        
        # R-Drop: 2 forward passes
        logits1 = model(input_ids, attention_mask)
        logits2 = model(input_ids, attention_mask)
        
        # Cross entropy loss
        ce_loss = (criterion(logits1, labels) + criterion(logits2, labels)) / 2
        
        # KL divergence loss
        kl_loss = compute_kl_loss(logits1, logits2)
        
        # Total loss
        loss = ce_loss + rdrop_alpha * kl_loss
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        
        optimizer.step()
        scheduler.step()
        
        total_loss += loss.item()
        avg_logits = (logits1 + logits2) / 2
        preds = torch.argmax(avg_logits, dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.cpu().numpy())
        
        progress_bar.set_postfix({'loss': f'{loss.item():.4f}'})
    
    avg_loss = total_loss / len(dataloader)
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')
    
    return avg_loss, accuracy, f1

def evaluate(model, dataloader, criterion, device):
    """Evaluate model"""
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            
            logits = model(input_ids, attention_mask)
            loss = criterion(logits, labels)
            
            total_loss += loss.item()
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())
    
    avg_loss = total_loss / len(dataloader)
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')
    
    return avg_loss, accuracy, f1, all_preds, all_labels

class EarlyStopping:
    """Early stopping dengan gap monitoring"""
    
    def __init__(self, patience=7, min_delta=0.001, mode='max', max_gap=0.08):
        self.patience = patience
        self.min_delta = min_delta
        self.mode = mode
        self.max_gap = max_gap
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.best_model = None
        self.best_gap = float('inf')
    
    def __call__(self, score, model, train_score=None):
        if self.mode == 'max':
            is_improvement = self.best_score is None or score > self.best_score + self.min_delta
        else:
            is_improvement = self.best_score is None or score < self.best_score - self.min_delta
        
        if is_improvement:
            self.best_score = score
            self.best_model = copy.deepcopy(model.state_dict())
            self.counter = 0
            if train_score is not None:
                self.best_gap = train_score - score
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        
        return self.early_stop

print('✓ Training functions defined')

## 🚀 9. Training Loop

In [ ]:
# Training history
history = {
    'train_loss': [], 'train_acc': [], 'train_f1': [],
    'val_loss': [], 'val_acc': [], 'val_f1': [],
    'gap': [],
    'epoch_time': [],
    'memory_used_mb': [],
    'power_draw_w': []
}

early_stopping = EarlyStopping(
    patience=CONFIG['early_stopping_patience'], 
    mode='max',
    max_gap=0.08
)

print('=' * 60)
print('🚀 TRAINING STARTED')
print('=' * 60)
print(f'Device: {device}')
print(f'Epochs: {CONFIG["epochs"]} | Patience: {CONFIG["early_stopping_patience"]}')
print(f'LR: {CONFIG["learning_rate"]} | Batch: {CONFIG["batch_size"]}')
print(f'Frozen Layers: {CONFIG["freeze_layers"]}/12 | Dropout: {CONFIG["dropout_rate"]}')
print('-' * 60)

# Show initial GPU stats
if torch.cuda.is_available():
    gpu_stats = get_gpu_stats()
    if gpu_stats:
        print(f'🎮 GPU: {torch.cuda.get_device_name(0)}')
        print(f'   Memory: {gpu_stats["memory_used_mb"]:.0f}/{gpu_stats["memory_total_mb"]:.0f} MB')
        print(f'   Power: {gpu_stats["power_draw_w"]:.1f} W | Temp: {gpu_stats["temperature_c"]:.0f}°C')
print('-' * 60)

best_val_f1 = 0
best_epoch = 0
best_gap = float('inf')

# Start GPU monitoring
gpu_monitor.start()
training_start_time = datetime.now()

for epoch in range(CONFIG['epochs']):
    epoch_start = time.time()
    print(f'\n📍 Epoch {epoch + 1}/{CONFIG["epochs"]}')
    
    # Train
    train_loss, train_acc, train_f1 = train_epoch(
        model, train_loader, criterion, optimizer, scheduler, 
        device, CONFIG['max_grad_norm'], CONFIG['rdrop_alpha']
    )
    
    # Sample GPU stats after training
    gpu_monitor.sample()
    
    # Validate
    val_loss, val_acc, val_f1, _, _ = evaluate(
        model, val_loader, criterion, device
    )
    
    gap = train_acc - val_acc
    epoch_time = time.time() - epoch_start
    
    # Get current GPU stats
    current_memory = 0
    current_power = 0
    if torch.cuda.is_available():
        current_memory = torch.cuda.memory_allocated(0) / 1024**2
        current_power = get_gpu_power()
    
    # Save history
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['train_f1'].append(train_f1)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    history['val_f1'].append(val_f1)
    history['gap'].append(gap)
    history['epoch_time'].append(epoch_time)
    history['memory_used_mb'].append(current_memory)
    history['power_draw_w'].append(current_power)
    
    # Print metrics
    print(f'  Train - Loss: {train_loss:.4f} | Acc: {train_acc:.4f} | F1: {train_f1:.4f}')
    print(f'  Val   - Loss: {val_loss:.4f} | Acc: {val_acc:.4f} | F1: {val_f1:.4f}')
    
    # Track best
    if val_f1 > best_val_f1 and gap < 0.10:
        best_val_f1 = val_f1
        best_epoch = epoch + 1
        best_gap = gap
        print(f'  ⭐ New best! F1: {val_f1:.4f}, Gap: {gap*100:.2f}%')
    
    # Gap status
    gap_status = '✅ Good' if gap < 0.05 else '⚡ OK' if gap < 0.10 else '⚠️ High'
    print(f'  📊 Gap: {gap*100:.2f}% {gap_status}')
    print(f'  ⏱️  Time: {epoch_time:.1f}s | 💾 Mem: {current_memory:.0f}MB | ⚡ Power: {current_power:.1f}W')
    
    # Early stopping
    if early_stopping(val_f1, model, train_acc):
        print(f'\n🛑 Early stopping at epoch {epoch + 1}')
        break

# Stop GPU monitoring
gpu_monitor.stop()
training_end_time = datetime.now()
total_time = training_end_time - training_start_time

# Load best model
if early_stopping.best_model is not None:
    model.load_state_dict(early_stopping.best_model)

# Get GPU summary
gpu_summary = gpu_monitor.get_summary()

print('\n' + '=' * 60)
print('✅ TRAINING COMPLETED')
print('=' * 60)
print(f'  Best epoch: {best_epoch}')
print(f'  Best Val F1: {best_val_f1:.4f}')
print(f'  Best Gap: {best_gap*100:.2f}%')
print()
print('⏱️  TIME STATISTICS:')
print(f'  Total Duration: {gpu_summary["duration_formatted"]}')
print(f'  Avg Time/Epoch: {np.mean(history["epoch_time"]):.1f}s')
print()
print('💾 MEMORY STATISTICS:')
print(f'  Peak Memory: {gpu_summary["peak_memory_mb"]:.0f} MB')
print(f'  Avg Memory: {gpu_summary["avg_memory_mb"]:.0f} MB')
print()
print('⚡ POWER STATISTICS:')
print(f'  Avg Power: {gpu_summary["avg_power_w"]:.1f} W')
print(f'  Max Power: {gpu_summary["max_power_w"]:.1f} W')
print(f'  Total Energy: {gpu_summary["total_energy_wh"]:.2f} Wh ({gpu_summary["total_energy_kwh"]:.4f} kWh)')
print('=' * 60)

## 📈 10. Training Visualization & Resource Usage

In [ ]:
# =====================================================
# FIGURE 1: Training Metrics
# =====================================================
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
epochs_range = range(1, len(history['train_loss']) + 1)

# Loss
axes[0, 0].plot(epochs_range, history['train_loss'], 'b-o', label='Train', markersize=4)
axes[0, 0].plot(epochs_range, history['val_loss'], 'r-s', label='Val', markersize=4)
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].set_title('Training & Validation Loss')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Accuracy
axes[0, 1].plot(epochs_range, history['train_acc'], 'b-o', label='Train', markersize=4)
axes[0, 1].plot(epochs_range, history['val_acc'], 'r-s', label='Val', markersize=4)
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Accuracy')
axes[0, 1].set_title('Training & Validation Accuracy')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# F1 Score
axes[1, 0].plot(epochs_range, history['train_f1'], 'b-o', label='Train', markersize=4)
axes[1, 0].plot(epochs_range, history['val_f1'], 'r-s', label='Val', markersize=4)
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('F1 Score')
axes[1, 0].set_title('Training & Validation F1 Score')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Gap
axes[1, 1].plot(epochs_range, [g*100 for g in history['gap']], 'g-o', markersize=4)
axes[1, 1].axhline(y=5, color='orange', linestyle='--', label='Good threshold (5%)')
axes[1, 1].axhline(y=10, color='red', linestyle='--', label='Warning threshold (10%)')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Train-Val Gap (%)')
axes[1, 1].set_title('Overfitting Monitor (Train-Val Gap)')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_history.png', dpi=150, bbox_inches='tight')
plt.show()
print('✓ Saved: training_history.png')

In [ ]:
# =====================================================
# FIGURE 2: Resource Usage (Time, Memory, Power)
# =====================================================
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
epochs_range = range(1, len(history['epoch_time']) + 1)

# Time per epoch
axes[0, 0].bar(epochs_range, history['epoch_time'], color='steelblue', alpha=0.7)
axes[0, 0].axhline(y=np.mean(history['epoch_time']), color='red', linestyle='--', 
                   label=f'Avg: {np.mean(history["epoch_time"]):.1f}s')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Time (seconds)')
axes[0, 0].set_title('⏱️ Training Time per Epoch')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3, axis='y')

# Memory usage
if any(history['memory_used_mb']):
    axes[0, 1].plot(epochs_range, history['memory_used_mb'], 'purple', marker='o', markersize=4)
    axes[0, 1].axhline(y=gpu_summary['peak_memory_mb'], color='red', linestyle='--', 
                       label=f'Peak: {gpu_summary["peak_memory_mb"]:.0f} MB')
    axes[0, 1].fill_between(epochs_range, 0, history['memory_used_mb'], alpha=0.3, color='purple')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Memory (MB)')
    axes[0, 1].set_title('💾 GPU Memory Usage')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
else:
    axes[0, 1].text(0.5, 0.5, 'No GPU Memory Data', ha='center', va='center', fontsize=14)
    axes[0, 1].set_title('💾 GPU Memory Usage')

# Power consumption
if any(history['power_draw_w']):
    axes[1, 0].plot(epochs_range, history['power_draw_w'], 'orange', marker='s', markersize=4)
    axes[1, 0].axhline(y=gpu_summary['avg_power_w'], color='red', linestyle='--', 
                       label=f'Avg: {gpu_summary["avg_power_w"]:.1f} W')
    axes[1, 0].fill_between(epochs_range, 0, history['power_draw_w'], alpha=0.3, color='orange')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Power (Watts)')
    axes[1, 0].set_title('⚡ GPU Power Consumption')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
else:
    axes[1, 0].text(0.5, 0.5, 'No Power Data', ha='center', va='center', fontsize=14)
    axes[1, 0].set_title('⚡ GPU Power Consumption')

# Summary statistics as text
axes[1, 1].axis('off')
summary_text = f"""
╔══════════════════════════════════════════════╗
║         📊 TRAINING RESOURCE SUMMARY         ║
╠══════════════════════════════════════════════╣
║                                              ║
║  ⏱️  TIME STATISTICS                         ║
║  ├─ Total Duration: {gpu_summary['duration_formatted']:>20}  ║
║  ├─ Total Seconds:  {gpu_summary['duration_seconds']:>17.1f}s  ║
║  └─ Avg per Epoch:  {np.mean(history['epoch_time']):>17.1f}s  ║
║                                              ║
║  💾 MEMORY STATISTICS                        ║
║  ├─ Peak Memory:    {gpu_summary['peak_memory_mb']:>15.0f} MB  ║
║  └─ Avg Memory:     {gpu_summary['avg_memory_mb']:>15.0f} MB  ║
║                                              ║
║  ⚡ POWER STATISTICS                         ║
║  ├─ Avg Power:      {gpu_summary['avg_power_w']:>16.1f} W  ║
║  ├─ Max Power:      {gpu_summary['max_power_w']:>16.1f} W  ║
║  ├─ Total Energy:   {gpu_summary['total_energy_wh']:>14.2f} Wh  ║
║  └─ Total Energy:   {gpu_summary['total_energy_kwh']:>12.4f} kWh  ║
║                                              ║
╚══════════════════════════════════════════════╝
"""
axes[1, 1].text(0.1, 0.5, summary_text, fontsize=11, fontfamily='monospace',
                verticalalignment='center', bbox=dict(boxstyle='round', facecolor='lightgray', alpha=0.8))

plt.tight_layout()
plt.savefig('resource_usage.png', dpi=150, bbox_inches='tight')
plt.show()
print('✓ Saved: resource_usage.png')

## 🧪 11. Evaluation on Test Set

In [ ]:
# Evaluate on test set
test_loss, test_acc, test_f1, test_preds, test_labels = evaluate(
    model, test_loader, criterion, device
)

print('=' * 60)
print('🧪 TEST SET EVALUATION')
print('=' * 60)
print(f'Test Accuracy: {test_acc*100:.2f}%')
print(f'Test F1 Score: {test_f1*100:.2f}%')
print(f'Test Loss: {test_loss:.4f}')

# Classification report
print('\n📊 Classification Report:')
print(classification_report(test_labels, test_preds, target_names=LABEL_NAMES))

In [ ]:
# Confusion Matrix
cm = confusion_matrix(test_labels, test_preds)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Counts
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=LABEL_NAMES, yticklabels=LABEL_NAMES, ax=axes[0])
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('Actual')
axes[0].set_title('Confusion Matrix (Counts)')

# Percentages
sns.heatmap(cm_normalized, annot=True, fmt='.1f', cmap='Blues',
            xticklabels=LABEL_NAMES, yticklabels=LABEL_NAMES, ax=axes[1])
axes[1].set_xlabel('Predicted')
axes[1].set_ylabel('Actual')
axes[1].set_title('Confusion Matrix (%)')

plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=150, bbox_inches='tight')
plt.show()
print('✓ Saved: confusion_matrix.png')

## 💾 12. Save Model

In [ ]:
# Create models directory
os.makedirs('models', exist_ok=True)

# Save model
model_path = 'models/indobert_sentiment_3class.pt'
torch.save({
    'model_state_dict': model.state_dict(),
    'config': CONFIG,
    'label_map': LABEL_MAP,
    'label_names': LABEL_NAMES,
    'test_accuracy': test_acc,
    'test_f1': test_f1,
    'best_val_f1': best_val_f1,
    'best_gap': best_gap,
    'history': history,
}, model_path)
print(f'✓ Model saved: {model_path}')

# Save tokenizer
tokenizer.save_pretrained('models/tokenizer')
print(f'✓ Tokenizer saved: models/tokenizer/')

# Save history
with open('models/training_history.json', 'w') as f:
    json.dump(history, f, indent=2)
print(f'✓ History saved: models/training_history.json')

# List saved files
print('\n📁 Saved files:')
for root, dirs, files in os.walk('models'):
    for file in files:
        filepath = os.path.join(root, file)
        size = os.path.getsize(filepath) / (1024*1024)
        print(f'   {filepath} ({size:.2f} MB)')

## 🔮 13. Inference Demo

In [ ]:
def predict_sentiment(text, model, tokenizer, device, label_names):
    """Predict sentiment untuk satu teks"""
    model.eval()
    
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    with torch.no_grad():
        logits = model(input_ids, attention_mask)
        probs = torch.softmax(logits, dim=1)
        pred = torch.argmax(probs, dim=1).item()
    
    return {
        'sentiment': label_names[pred],
        'confidence': probs[0][pred].item(),
        'probabilities': {
            label_names[i]: probs[0][i].item() 
            for i in range(len(label_names))
        }
    }

# Test
test_reviews = [
    "Aplikasi gojek sangat membantu, driver ramah dan cepat",
    "Driver nya lama banget, udah nunggu 1 jam gak datang",
    "Biasa aja sih aplikasinya",
    "Pelayanan buruk, tidak akan pakai lagi",
    "Mantap, makanan sampai dengan selamat dan masih hangat",
]

print('=' * 60)
print('🔮 INFERENCE DEMO')
print('=' * 60)

for review in test_reviews:
    result = predict_sentiment(review, model, tokenizer, device, LABEL_NAMES)
    emoji = {'negative': '😠', 'neutral': '😐', 'positive': '😊'}[result['sentiment']]
    print(f'\n📝 "{review[:50]}..."' if len(review) > 50 else f'\n📝 "{review}"')
    print(f'   {emoji} {result["sentiment"].upper()} ({result["confidence"]*100:.1f}%)')

## 📊 14. Final Summary

In [ ]:
final_gap = history['train_acc'][-1] - history['val_acc'][-1]
min_gap = min(history['gap'])

print('=' * 70)
print('📊 FINAL TRAINING SUMMARY')
print('=' * 70)

print(f'''
🎯 MODEL PERFORMANCE:
   • Test Accuracy: {test_acc*100:.2f}%
   • Test F1 Score: {test_f1*100:.2f}%
   • Best Val F1: {best_val_f1*100:.2f}%

📈 OVERFITTING CHECK:
   • Final Gap: {final_gap*100:.2f}%
   • Best Gap: {best_gap*100:.2f}%
   • Status: {"✅ Good" if final_gap < 0.05 else "⚠️ Check" if final_gap < 0.10 else "❌ Overfitting"}

⏱️  TIME & RESOURCE USAGE:
   • Total Training Time: {gpu_summary['duration_formatted']}
   • Average Time per Epoch: {np.mean(history['epoch_time']):.1f} seconds
   • Peak GPU Memory: {gpu_summary['peak_memory_mb']:.0f} MB
   • Average GPU Memory: {gpu_summary['avg_memory_mb']:.0f} MB
   • Average Power Draw: {gpu_summary['avg_power_w']:.1f} W
   • Max Power Draw: {gpu_summary['max_power_w']:.1f} W
   • Total Energy Consumed: {gpu_summary['total_energy_wh']:.2f} Wh ({gpu_summary['total_energy_kwh']:.4f} kWh)

⚙️ TECHNIQUES USED:
   • Layer Freezing: {CONFIG['freeze_layers']}/12
   • Dropout: {CONFIG['dropout_rate']}
   • R-Drop Alpha: {CONFIG['rdrop_alpha']}
   • Weight Decay: {CONFIG['weight_decay']}
   • Label Smoothing: {CONFIG['label_smoothing']}
   • Learning Rate: {CONFIG['learning_rate']}

💾 SAVED FILES:
   • models/indobert_sentiment_3class.pt
   • models/tokenizer/
   • models/training_history.json
   • training_history.png
   • resource_usage.png
   • confusion_matrix.png
''')

if test_acc >= 0.75 and final_gap < 0.05:
    print('🎉 EXCELLENT! Model has good accuracy and generalization!')
elif test_acc >= 0.75:
    print('⚠️ Good accuracy but watch for overfitting.')
elif final_gap < 0.05:
    print('✅ Good generalization but accuracy could improve.')
else:
    print('❌ Consider adjusting hyperparameters.')

print('=' * 70)
print('✅ Training completed!')
print('=' * 70)

# Save complete training report
training_report = {
    'model_performance': {
        'test_accuracy': float(test_acc),
        'test_f1_score': float(test_f1),
        'best_val_f1': float(best_val_f1),
        'best_epoch': best_epoch,
        'final_gap': float(final_gap),
        'best_gap': float(best_gap)
    },
    'resource_usage': {
        'total_duration_seconds': gpu_summary['duration_seconds'],
        'total_duration_formatted': gpu_summary['duration_formatted'],
        'avg_epoch_time_seconds': float(np.mean(history['epoch_time'])),
        'peak_memory_mb': gpu_summary['peak_memory_mb'],
        'avg_memory_mb': gpu_summary['avg_memory_mb'],
        'avg_power_watts': gpu_summary['avg_power_w'],
        'max_power_watts': gpu_summary['max_power_w'],
        'total_energy_wh': gpu_summary['total_energy_wh'],
        'total_energy_kwh': gpu_summary['total_energy_kwh']
    },
    'config': CONFIG,
    'history': history
}

with open('models/training_report.json', 'w') as f:
    json.dump(training_report, f, indent=2)
print('\n✓ Complete training report saved: models/training_report.json')